# <font color="red"> MBA em IA e Big Data</font>
## <span style="color:red">Linguagens e Ferramentas para Inteligência Artificial e Big Data (Python e SQL)</span>

### <span style="color:darkred">Importando e processando dados do Oracle via Python</span>

*Jose Fernando Rodrigues Junior*<br>
*ICMC/USP São Carlos*

Obs.: este notebook depende de softwares muito específicos que não funcionam em plataformas de cloud, como o Google Colab. Por esta razão, é preciso executá-lo localmente em sua máquina.

Devido à complexidade para se reproduzir este notebook, sua reprodução fica como ativididade OPCIONAL, recomendada apenas a usuários avançados.

# Instalação Windows - PARTE 1 - Anaconda com Jupyter Notebook

* 1) instale o software Anaconda3 (https://www.anaconda.com/) para ter funcionalidades python e notebooks via jupyter
* 2) copie este notebook (arquivo .ipynb) para o diretório c:\users\seu_nome_de_usuario
* 3) no menu Iniciar, procure por Jupyter Notebook, e execute
* 4) uma página web se abrirá, clique no arquivo deste notebook
 


# Instalação Windows - PARTE 2 - PREPARAÇÃO ORACLE
* 0) Oracle via VPN: https://shorturl.at/cnwW2
* 1) na linha de comando (cmd), execute: conda install -c anaconda cx_oracle
* 2) Instalar bibliotecas do MS Visual Studio https://aka.ms/vs/16/release/vc_redist.x64.exe - instalacao direta, sem entraves
* 3) Download do instant client https://download.oracle.com/otn_software/nt/instantclient/1911000/instantclient-basiclite-windows.x64-19.11.0.0.0dbru.zip
* 4) Descompactar o .zip para o diretório c:\temp (ou outro diretório fácil de lembrar) de modo que o caminho c:\temp\instantclient_19_11 seja válido
* 5) Execute os seguintes comandos no Notebook
* os.environ["PATH"] =r"C:\temp\instantclient_19_11" + ";" + os.environ["PATH"]
* cx_Oracle.init_oracle_client(lib_dir=r"C:\temp\instantclient_19_11")
* ATENCAO1: atencao à sintaxe lib_dir=r"C:\temp\instantclient_19_11", com "r" antes do caminho
* ATENCAO2: eu testei com o diretório c:\temp, substitua por seu diretório
* ATENCAO3: funciona com python 3

In [1]:
import cx_Oracle #documentacao https://cx-oracle.readthedocs.io/en/latest/user_guide/introduction.html
import pandas as pd
import sqlalchemy
import os

* O `cx_Oracle` é um módulo de extensão Python que permite acesso ao banco de dados Oracle; o cx_Oracle está, parcialmente, em conformidade com a especificação da `DB-API 2.0` para banco de dados por meio de Python;
* A python DB-API é uma especificação para uma interface comum para bancos de dados relacionais definida no documento `Python Enhancement Proposal 249` (https://www.python.org/dev/peps/pep-0249/);
* `SQLAlchemy` é um módulo Python com dois objetivos:
    * Kit de ferramentas Python-SQL: oferece funcionalidades flexíveis para uso simplificado e abstrato de SQL;
    * Mapeador Relacional de Objetos: uma técnica de mapeamento objeto relacional que permite fazer uma relação dos objetos programados com os dados relacionais que os mesmos representams.

In [2]:
os.environ["PATH"] =r"C:\temp\instantclient_19_11" + ";" + os.environ["PATH"]
try:
    cx_Oracle.init_oracle_client(lib_dir=r"C:\temp\instantclient_19_11")
except Exception as e:
    print(str(e))

In [7]:
#Caso esteja usando o Oracle Express Edition localmente, use o comando
#dsnStr = cx_Oracle.makedsn(host="localhost", port="1521", sid="xe")
dsnStr = cx_Oracle.makedsn(host="orclgrad1.icmc.usp.br", port="1521", service_name="pdb_junior.icmc.usp.br")

conexao = cx_Oracle.connect(user="M_SEU_CPF", password="SUA_SENHA", dsn=dsnStr)
print(conexao.version)

#No caso de PostgreSQL, usa-se a biblioteca psycopg2
#import psycopg2
#conn = psycopg2.connect(host=host_address, database=name_of_database, user=user_name, password=user_password)

19.3.0.0.0


## Cursor
* Uma conexão fornece objetos do tipo `cursor`; os quais permitem operações DDL e DML no banco conectado.

In [9]:
cursorDDL = conexao.cursor()
cursorDML = conexao.cursor()

## Criação de um esquema a partir de um script sql

    1) Aqui fazemos a leitura completa de um arquivo;
    2) Quebra do texto usando a operação de split, considerando o ";" como separador;
    3) Para cada comando encontrado, executamos o SQL por meio de um cursor;
    4) Tratamos eventuais erros usando uma construção try-except.

In [11]:
#Primeiro o esquema
print("Certifique-se que o diretório EsquemaFutebol está dentro do diretório corrente:")
print(os.getcwd())
print()
fd = open('./EsquemaFutebol/Esquema_Futebol.sql', 'r')
sqlFile = fd.read()
fd.close()

sqlCommands = sqlFile.split(';')
for command in sqlCommands:
    if(len(command) < 3): break;
    try:
        print(command)
        print()
        cursorDDL.execute(command)
    except Exception as msg:
        print("Erro de SQL: "+ str(msg) + ": "+ command)
print('Script finalizado.')

Certifique-se que o diretório EsquemaFutebol está dentro do diretório corrente:
c:\Ju\USP\MBA-IA_BIG_DATA\Curso01\2023

Na primeira vez, execute esta célula duas vezes.

Drop table time cascade constraints


Drop table joga cascade constraints


Drop table partida cascade constraints


Drop table jogador cascade constraints


Drop table posicao_jogador cascade constraints


Drop table diretor cascade constraints


Drop table uniforme cascade constraints




CREATE TABLE time (
      nome       VARCHAR2(40) NOT NULL,
      estado     CHAR(2),
      tipo       VARCHAR2(15),
      saldo_gols INTEGER,
      
      CONSTRAINT pk_time PRIMARY KEY (nome),
      CONSTRAINT ck_tipo CHECK (tipo in ('amador', 'profissional'))
)



CREATE TABLE joga (
      time1      VARCHAR2(40) NOT NULL,
      time2      VARCHAR2(40) NOT NULL,
      classico   CHAR(1),

      CONSTRAINT pk_joga PRIMARY KEY (time1, time2),
      CONSTRAINT fk_joga1 FOREIGN KEY (time1) REFERENCES time(NOME) ON DELETE CASCADE,
   

In [12]:
#Agora as relações instanciadas
fd = open('./EsquemaFutebol/Dados_Futebol.sql', 'r')
sqlFile = fd.read()
fd.close()

sqlCommands = sqlFile.split(';')
for command in sqlCommands:
    if(len(command) < 3): break;    
    try:
        print(command)
        print()
        cursorDDL.execute(command)        
    except Exception as msg:
        print("Erro de SQL: "+ str(msg) + ": |"+ command+'|')
print('Script finalizado.')

insert into time values ('Sao Paulo',     'SP', 'profissional', 8)


insert into time values ('Palmeiras',     'SP', 'profissional', 5)


insert into time values ('Santos',        'SP', 'profissional', 0)


insert into time values ('Corinthians',   'SP', 'profissional', 6)


insert into time values ('Paulistinha',   'SP', 'amador',       1)


insert into time values ('Ibate',         'SP', 'amador',       0)


insert into time values ('Cruzeiro',      'MG', 'profissional', 2)


insert into time values ('Atletico',      'MG', 'profissional', 3)


insert into time values ('Frutal',        'MG', 'amador',       1)



insert into joga values ('Sao Paulo',   'Palmeiras',   'S')


insert into joga values ('Ibate',       'Paulistinha', 'N')


insert into joga values ('Ibate',       'Frutal',      'S')


insert into joga values ('Santos',      'Corinthians', 'N')


insert into joga values ('Corinthians', 'Palmeiras',   'S')


insert into joga values ('Santos',      'Sao Paulo',    'N')


inser

In [13]:
#os dados soh aparecerao para outros clientes do banco apos este comando
cursorDDL.execute('commit')

# Consulta à estrutura do esquema por meio do Dicionário de Dados

* O dicionário de dados é o banco de dados que guarda informações sobre os esquemas presentes no banco de dados.
Ele é usado para a realização de meta operações sobre a base de dados; por exemplo, a interface gráfica de um cliente SQL, ou os campos que constituirão um formulário de dados.

In [14]:
cursorDML.execute('select * from user_tables')
print('Lista das tabelas no esquema do usuário que estabeleceu a conexão:')
for tupla in cursorDML:     
    print(tupla[0])

Lista das tabelas no esquema do usuário que estabeleceu a conexão:
DIRETOR
JOGA
JOGADOR
LE01ESTADO
LE02CIDADE
LE03ZONA
LE04BAIRRO
LE05URNA
LE06SESSAO
LE07PARTIDO
LE08CANDIDATO
LE09CARGO
LE10CANDIDATURA
LE11PLEITO
LE12PESQUISA
LE13INTENCAODEVOTO
PARTIDA
POSICAO_JOGADOR
REF
SESSAO
TESTE
TESTE10
TIME
UNIFORME
URNA


* Aqui, define-se uma função para recuperar todas as chaves estrangeiras de uma dada tabela.

In [15]:
def printForeignKeys(sATableName = ''):
    SQL=('SELECT a.table_name, a.column_name, a.constraint_name, c.owner, c.r_owner, '
         'c_pk.table_name r_table_name, c_pk.constraint_name r_pk FROM user_cons_columns a '
         'JOIN user_constraints c ON a.owner = c.owner '
         'AND a.constraint_name = c.constraint_name '
         'JOIN user_constraints c_pk ON c.r_owner = c_pk.owner '
         'AND c.r_constraint_name = c_pk.constraint_name '
         'WHERE c.constraint_type = \'R\' ')
    if(sATableName != ''):
        SQL += 'AND a.table_name = \'' + sATableName.upper() +'\' '
    SQL += 'ORDER BY a.table_name, constraint_name'
    cursorTemp = conexao.cursor()
    cursorTemp.execute(SQL)
    for tupla in cursorTemp:
        print(tupla)
    cursorTemp.close()

In [17]:
printForeignKeys('') #este comando pode demorar um pouco

('DIRETOR', 'TIMEDIR', 'FK_DIRETOR', 'D2373891', 'D2373891', 'TIME', 'PK_TIME')
('JOGA', 'TIME1', 'FK_JOGA1', 'D2373891', 'D2373891', 'TIME', 'PK_TIME')
('JOGA', 'TIME2', 'FK_JOGA2', 'D2373891', 'D2373891', 'TIME', 'PK_TIME')
('JOGADOR', 'TIME_ATUA', 'FK_JOGADOR', 'D2373891', 'D2373891', 'TIME', 'PK_TIME')
('LE02CIDADE', 'SIGLAESTADO', 'FKESTADOCIDADE', 'D2373891', 'D2373891', 'LE01ESTADO', 'PKESTADO')
('LE04BAIRRO', 'SIGLAESTADO', 'FKCIDADEBAIRRO', 'D2373891', 'D2373891', 'LE02CIDADE', 'PKCIDADE')
('LE04BAIRRO', 'NOMECIDADE', 'FKCIDADEBAIRRO', 'D2373891', 'D2373891', 'LE02CIDADE', 'PKCIDADE')
('LE04BAIRRO', 'NROZONA', 'FKZONABAIRRO', 'D2373891', 'D2373891', 'LE03ZONA', 'PKZONA')
('LE05URNA', 'NROZONA', 'FKZONAURNA', 'D2373891', 'D2373891', 'LE03ZONA', 'PKZONA')
('LE06SESSAO', 'NSERIAL', 'FKURNASESSAO', 'D2373891', 'D2373891', 'LE05URNA', 'PKURNA')
('LE06SESSAO', 'NROZONA', 'FKZONASESSAO', 'D2373891', 'D2373891', 'LE03ZONA', 'PKZONA')
('LE08CANDIDATO', 'SIGLAPARTIDO', 'FKPARTIDOCANDIDA

## Diagrama da base de dados Futebol, usada neste notebook
(execute a celula abaixo para visualizar)

<img src="./EsquemaFutebol/EsquemaFutebol.png" width=350 height=350 />

* Inserindo alguns dados.

In [18]:
cursorDML.execute("insert into Time values ('Fortaleza', 'CE', 'profissional', 2)")
#usando binding variables
cursorDML.execute("insert into Time values (:a,:b,:c,:d)",['Juventude', 'RS', 'profissional', 3])
cursorDML.execute("insert into Joga values (:a,:b,:c)",['Juventude', 'Fortaleza', 'N'])

* Se foi tudo ok, executar `commit()`; do contrário, executar `rollback()` para desfazer as atualizações

In [19]:
conexao.commit()

# Recuperando dados com cursores
* Pode-se requisitar os dados da base um por vez, o que consome mais recursos de rede e processamento no servidor; ou múltiplos por vez (batch), o que é mais eficiente. Se houver memória (ou poucos dados) pode-se recuperar todos os dados de uma vez.

* Um por vez

In [20]:
cursorDML.execute("select * from Joga where classico = \'N\'")
while True:
    row = cursorDML.fetchone()  #Um por vez
    if row is None: break
    print(row)

('Ibate', 'Paulistinha', 'N')
('Santos', 'Corinthians', 'N')
('Santos', 'Sao Paulo', 'N')
('Santos', 'Palmeiras', 'N')
('Juventude', 'Fortaleza', 'N')


* **Batch**: menos requisições de rede/transferência de dados; menos processamento no servidor

In [21]:
cursorDML.execute("select * from Time")
num_rows = 11
while True:
    rows = cursorDML.fetchmany(num_rows) #Muitos por vez
    if not rows: break
    for row in rows:
        print(row)
        
print()
print("Foram recuperadas "+str(cursorDML.rowcount) +" tuplas.")


('Sao Paulo', 'SP', 'profissional', 8)
('Palmeiras', 'SP', 'profissional', 5)
('Santos', 'SP', 'profissional', 0)
('Corinthians', 'SP', 'profissional', 6)
('Paulistinha', 'SP', 'amador', 1)
('Ibate', 'SP', 'amador', 0)
('Cruzeiro', 'MG', 'profissional', 2)
('Atletico', 'MG', 'profissional', 3)
('Frutal', 'MG', 'amador', 1)
('Fortaleza', 'CE', 'profissional', 2)
('Juventude', 'RS', 'profissional', 3)

Foram recuperadas 11 tuplas.


* É possível ainda recuperar meta-informações sobre as colunas retornadas por meio do cursor

In [22]:
for metadata in cursorDML.description:
    print(metadata)

('NOME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 40, 40, None, None, 0)
('ESTADO', <cx_Oracle.DbType DB_TYPE_CHAR>, 2, 2, None, None, 1)
('TIPO', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 15, 15, None, None, 1)
('SALDO_GOLS', <cx_Oracle.DbType DB_TYPE_NUMBER>, 39, None, 38, 0, 1)


# Usando rowfactories

* Com rowfactories podemos processar os dados recuperados antes de retorná-los.

In [32]:
cursorDML.execute("select * from jogador")
# col[0] eh o nome de cada atributo
# o comando a seguir monta o array (lista) com os nomes dos atributos
columns = [col[0] for col in cursorDML.description]

# aqui definimos como o rowfactory serah executado quando um comando de fecth acontecer
cursorDML.rowfactory = lambda *args: zip(columns, args) #aplica lambda para cada tupla


In [33]:
# a operação de fetch garante a execução do lambda
data = cursorDML.fetchone()
# o resultado é o produto cartesiano {nomes do atributos} x {valores dos atributos}
print(list(data))

[('RG', '111111111'), ('NOME', 'Pele'), ('DATA_NASCIMENTO', datetime.datetime(1955, 5, 15, 0, 0)), ('NATURALIDADE', 'Santos'), ('TIME_ATUA', 'Santos')]


# Lendo dados do banco diretamente para um Pandas Dataframe

* Uma consulta SQL.

In [34]:
meuDataFrame = pd.read_sql_query('SELECT * FROM Partida', con=conexao)

In [35]:
meuDataFrame

,TIME1,TIME2,DATA,LOCAL,PLACAR
0,Sao Paulo,Palmeiras,2007-05-15,Morumbi,2x0
1,Santos,Sao Paulo,2007-05-20,Pacaembu,1x0
2,Santos,Palmeiras,2007-06-06,Vila Belmiro,0x1
3,Ibate,Paulistinha,2007-05-25,Luizao,0x0
4,Santos,Corinthians,2007-05-30,Pacaembu,0x0


* ATENCAO: um dataframe nada mais é do que `uma tabela (uma relação) em memória`. Muito provavelmente, ler uma tabela inteira em memória vai causar problemas de falta de memória e de processamento. Uma solução é ler pedaços (**chunks**) da tabela em um objeto do tipo **generator**;

* Pode-se fazer isso usando-se uma variação do `read_sql_query()` com o parâmetro `chunksize` fornecido. Neste caso, o resultado não será um DataFrame, mas sim um objeto do tipo **generator**, o qual fornecerá um DataFrame com chunksize elementos a cada iteração;

* **Generator**: uma abstração de dados que permite a geração de resultados/produtos sequencialmente, os quais são produzidos mediante requisição. Após um generator ser consumido, ele precisa ser regerado.

In [36]:
dataFrameGenerator = pd.read_sql_query('SELECT * FROM Time', con=conexao, chunksize=3)
type(dataFrameGenerator)

generator

In [37]:
for i, dataFrameChunk in enumerate(dataFrameGenerator):
    print('-'*10)
    print("Chunk "+str(i))
    print(dataFrameChunk)


----------
Chunk 0
        NOME ESTADO          TIPO  SALDO_GOLS
0  Sao Paulo     SP  profissional           8
1  Palmeiras     SP  profissional           5
2     Santos     SP  profissional           0
----------
Chunk 1
          NOME ESTADO          TIPO  SALDO_GOLS
0  Corinthians     SP  profissional           6
1  Paulistinha     SP        amador           1
2        Ibate     SP        amador           0
----------
Chunk 2
       NOME ESTADO          TIPO  SALDO_GOLS
0  Cruzeiro     MG  profissional           2
1  Atletico     MG  profissional           3
2    Frutal     MG        amador           1
----------
Chunk 3
        NOME ESTADO          TIPO  SALDO_GOLS
0  Fortaleza     CE  profissional           2
1  Juventude     RS  profissional           3


* Com o processamento em chunks é possível executar processamento pedaço por pedaço do dataset.

In [38]:
total_de_gols = 0
dataFrameGenerator = pd.read_sql_query('SELECT * FROM time', con=conexao, chunksize=3)

In [39]:
for i, dataFrameChunk in enumerate(dataFrameGenerator):
    total_chunk = dataFrameChunk['SALDO_GOLS'].sum()
    print('Total de gols do ' + str(i) + '-esimo chunk: '+str(total_chunk))
    total_de_gols += total_chunk
print("Total de gols da tabela inteira: " + str(total_de_gols))

Total de gols do 0-esimo chunk: 13
Total de gols do 1-esimo chunk: 7
Total de gols do 2-esimo chunk: 6
Total de gols do 3-esimo chunk: 5
Total de gols da tabela inteira: 31


* ATENCAO: o processamento de agregações é muito eficiente em SGBDs relacionais. Trazer dados do database para a memória, computar, e depois totalizar só é recomendado para tarefas que não podem ser processadas, ou que são muito complexas, em um SGBD. Para o exemplo da totalização, basta um simples SQL:

In [40]:
for row in cursorDML.execute("Select SUM(saldo_gols) from time"): print(row[0])

31


# SQLAlchemy - SQL Expression Language

* O **SQLAlchemy** oferece um conjunto de ferramentas, o **SQL Expression Language**, que permitem executar SQL sem a necessidade de escrever código SQL;
* Documentação completa em https://docs.sqlalchemy.org/en/14/genindex.html#T

* Primeiro, cria-se um **engine** usando-se a string de conexão.

In [41]:
dsnStr

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=orclgrad1.icmc.usp.br)(PORT=1521))(CONNECT_DATA=(SERVICE_NAME=pdb_junior.icmc.usp.br)))'

In [43]:
ora_engine = sqlalchemy.create_engine(f'oracle+cx_oracle://M_SEU_CPF:SUA_SENHA@'+dsnStr, echo=True)
ora_engine.connect()

2023-07-07 14:06:38,688 INFO sqlalchemy.engine.Engine select sys_context( 'userenv', 'current_schema' ) from dual
2023-07-07 14:06:38,688 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-07-07 14:06:38,936 INFO sqlalchemy.engine.Engine SELECT value FROM v$parameter WHERE name = 'compatible'
2023-07-07 14:06:38,936 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-07-07 14:06:39,020 INFO sqlalchemy.engine.Engine select value from nls_session_parameters where parameter = 'NLS_NUMERIC_CHARACTERS'
2023-07-07 14:06:39,020 INFO sqlalchemy.engine.Engine [raw sql] {}


* Em seguida, cria-se um **metaobjeto** com as informações da tabela onde se deseja realizar operações.
* No exemplo a seguir, o objeto jogador guardará em memória todas as informações que descrevem o esquema da relação Jogador, assim como suas restrições de integridade. O SQLAlchemy exibe o acesso feito ao dicionário de dados.

In [44]:
metadata = sqlalchemy.MetaData()
jogador = sqlalchemy.Table('jogador', metadata, autoload_with=ora_engine)

2023-07-07 14:08:19,136 INFO sqlalchemy.engine.Engine SELECT table_name, compression, compress_for FROM ALL_TABLES WHERE table_name = :table_name AND owner = :owner 
2023-07-07 14:08:19,137 INFO sqlalchemy.engine.Engine [generated in 0.00054s] {'table_name': 'JOGADOR', 'owner': 'D2373891'}
2023-07-07 14:08:19,438 INFO sqlalchemy.engine.Engine 
            SELECT
                col.column_name,
                col.data_type,
                col.char_length,
                col.data_precision,
                col.data_scale,
                col.nullable,
                col.data_default,
                com.comments,
                col.virtual_column,
                                col.default_on_null,
                (
                    SELECT id.generation_type || ',' || id.IDENTITY_OPTIONS
                    FROM ALL_TAB_IDENTITY_COLS id
                    WHERE col.table_name = id.table_name
                    AND col.column_name = id.column_name
                    AND col.o

2023-07-07 14:08:29,513 INFO sqlalchemy.engine.Engine [cached since 9.54s ago] {'table_name': 'JOGADOR', 'owner': 'D2373891'}
2023-07-07 14:08:29,518 INFO sqlalchemy.engine.Engine 
            SELECT comments
            FROM all_tab_comments
            WHERE table_name = :table_name AND owner = :schema_name
        
2023-07-07 14:08:29,518 INFO sqlalchemy.engine.Engine [cached since 0.4124s ago] {'table_name': 'JOGADOR', 'schema_name': 'D2373891'}


In [45]:
# colunas
print(jogador.columns)

ImmutableColumnCollection(jogador.rg, jogador.nome, jogador.data_nascimento, jogador.naturalidade, jogador.time_atua)


In [47]:
# restricoes de integridade
for constr in jogador.constraints:
    print(constr)
    print()

PrimaryKeyConstraint(Column('rg', VARCHAR(length=15), table=<jogador>, primary_key=True, nullable=False))

ForeignKeyConstraint(<sqlalchemy.sql.base.DedupeColumnCollection object at 0x000001F3F1BA6A90>, None, name='fk_jogador', ondelete='SET NULL', link_to_name=True, table=Table('jogador', MetaData(), Column('rg', VARCHAR(length=15), table=<jogador>, primary_key=True, nullable=False), Column('nome', VARCHAR(length=40), table=<jogador>, nullable=False), Column('data_nascimento', DATE(), table=<jogador>), Column('naturalidade', VARCHAR(length=40), table=<jogador>), Column('time_atua', VARCHAR(length=40), ForeignKey('time.nome'), table=<jogador>), schema=None))



In [48]:
# restricoes de integridade referencial (chaves estrangeiras)
for constr in jogador.foreign_keys:
    print(constr)
    print()

ForeignKey('time.nome')



* A partir do meta objeto, é possível fazer requisições que montam SQL

In [49]:
insert_jogador = jogador.insert()

In [50]:
print(insert_jogador)

INSERT INTO jogador (rg, nome, data_nascimento, naturalidade, time_atua) VALUES (:rg, :nome, :data_nascimento, :naturalidade, :time_atua)


In [51]:
update_jogador = jogador.update()

In [52]:
print(update_jogador)

UPDATE jogador SET rg=:rg, nome=:nome, data_nascimento=:data_nascimento, naturalidade=:naturalidade, time_atua=:time_atua


In [53]:
delete_jogador = jogador.delete().where(jogador.c.time_atua=='Palmeiras')

In [56]:
# com o meta SQL delete_jogador preparado com um parametro fornecido, faz-se a preparacao
delete_jogador.compile().params

{'time_atua_1': 'Palmeiras'}

In [57]:
# depois, a execucao
result = ora_engine.execute(delete_jogador)

2023-07-07 14:11:52,219 INFO sqlalchemy.engine.Engine DELETE FROM jogador WHERE jogador.time_atua = :time_atua_1
2023-07-07 14:11:52,220 INFO sqlalchemy.engine.Engine [cached since 103.3s ago] {'time_atua_1': 'Palmeiras'}
2023-07-07 14:11:52,221 INFO sqlalchemy.engine.Engine COMMIT


* Para fazer a inserção, é necessário fornecer os valores que serão inseridos.

In [58]:
import datetime

In [59]:
insert_jogador = jogador.insert().values(rg='67891', nome='Edilson', data_nascimento=datetime.date(1980,9,20),
                                         naturalidade='Peruano',time_atua='Palmeiras')

In [60]:
print(insert_jogador)

INSERT INTO jogador (rg, nome, data_nascimento, naturalidade, time_atua) VALUES (:rg, :nome, :data_nascimento, :naturalidade, :time_atua)


* Os quais são processados com o comando `compile()`.

In [61]:
insert_jogador.compile().params

{'rg': '67891',
 'nome': 'Edilson',
 'data_nascimento': datetime.date(1980, 9, 20),
 'naturalidade': 'Peruano',
 'time_atua': 'Palmeiras'}

* E, finalmente, a execução.

In [62]:
result = ora_engine.execute(insert_jogador)

2023-07-07 14:12:21,699 INFO sqlalchemy.engine.Engine INSERT INTO jogador (rg, nome, data_nascimento, naturalidade, time_atua) VALUES (:rg, :nome, :data_nascimento, :naturalidade, :time_atua)
2023-07-07 14:12:21,700 INFO sqlalchemy.engine.Engine [generated in 0.00071s] {'rg': '67891', 'nome': 'Edilson', 'data_nascimento': datetime.date(1980, 9, 20), 'naturalidade': 'Peruano', 'time_atua': 'Palmeiras'}
2023-07-07 14:12:21,703 INFO sqlalchemy.engine.Engine COMMIT


* O resultado mostra meta informações a respeito da operação.

In [63]:
result.inserted_primary_key

('67891',)

In [64]:
conexao.commit()

In [65]:
conexao.close()

Neste notebook python, vimos como trazer dados do Oracle para a memoria, e manipulá-los com comandos python. Isto foi feito de 3 maneiras diferentes: com operacoes de fecth simples, com chunks de dados, e via SQLAlchemy.
Em futuras aulas/licoes, e em suas atividades práticas, este conteúdo será util para se processar dados de sua própria base sem a necessidade de arquivos e importações intermeriários.